# Lab 10 Convolution Neural Network

## Lab 10.1 Implementation of Simple CNN

1x28x28 input

Conv(5x5x5 Filter) -> 5x24x24

pool(5x12x12)

output(5x12x12)

In [7]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1,1,28,28)
conv1 = nn.Conv2d(1,5,5)
pool = nn.MaxPool2d(2)

In [9]:
out = conv1(inputs)
out2 = pool(out)
out.size()
out2.size()

torch.Size([1, 5, 12, 12])

## Lab 10.2 MNIST CNN

(Layer 1) Conv : in_c=1 out_c=32 k=3 st=1 pad=1

(Layer 1) MaxPool : k=2 st=2


(Layer 2) Conv : in_c=32 out_c=64 k=3 st=1 pad=1

(Layer 2) MaxPool : k=2 st=2


(View) batch_size x [7,7,64] to batch_size x [3136]

(dense) input=3136 out=10

In [113]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import visdom

import torch.nn.init
import torch.nn as nn
import torch.nn.functional as F

vis = visdom.Visdom()
vis.close(env="main")

''

In [114]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
            Y=loss_value,
            win = loss_plot,
            update = 'append'
            )

In [115]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [116]:
print(device)
torch.cuda.is_available()

cuda


True

In [117]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [118]:
#MNIST dataset
mnist_train = dsets.MNIST(root='dataset/MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='dataset/MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [119]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [120]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625,10,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [121]:
model = CNN().to(device)

value = torch.Tensor(1,1,28,28).to(device)
print((model(value)).shape)

torch.Size([1, 10])


In [122]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [123]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [124]:
#Training
total_batch = len(data_loader)
print('Learning Started')

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    print('Epoch:{:02d} cost = {}'.format(epoch+1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

Learning Started
Epoch:01 cost = 0.16126690804958344
Epoch:02 cost = 0.04264891892671585
Epoch:03 cost = 0.031033022329211235
Epoch:04 cost = 0.02271990291774273
Epoch:05 cost = 0.018666762858629227
Epoch:06 cost = 0.015003982000052929
Epoch:07 cost = 0.012411510571837425
Epoch:08 cost = 0.012570415623486042
Epoch:09 cost = 0.009091855958104134
Epoch:10 cost = 0.008686444722115993
Epoch:11 cost = 0.008385466411709785
Epoch:12 cost = 0.006233751308172941
Epoch:13 cost = 0.0066106137819588184
Epoch:14 cost = 0.006250510923564434
Epoch:15 cost = 0.006306073162704706
Learning Finished!


In [48]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ',accuracy.item())

Accuracy:  0.9908999800682068


## Lab 10.3 Visdom Example

In [49]:
import visdom
vis = visdom.Visdom()  ### 가능한 Visdom을 켜놓고 진행을 하자.

### Hello World!

In [51]:
vis.text("Hello, world!",env="main")

'window_376a9cc47427a8'

### Image

In [55]:
a=torch.randn(3,200,200)
vis.image(a,env="main")

'window_376a9d1811ba86'

In [58]:
vis.images(torch.Tensor(3,3,28,28),env="main")

'window_376a9d45f0bd92'

### Examples of MNIST and CIFAR 10

In [65]:
MNIST = dsets.MNIST(root='dataset/MNIST_data/',
                    train=True,
                    transform=transforms.ToTensor(),
                    download=True)

cifar10 = dsets.CIFAR10(root='dataset/cifar10/',
                      train=True,
                      transform=transforms.ToTensor(),
                      download=True)

170500096it [00:30, 16656457.90it/s]                                                                                   

#### CIFAR10

In [66]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_376a9e52b05646'

#### MNIST

In [68]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_376a9e790b302a'

### Check Dataset

In [69]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                         batch_size = 32,
                                         shuffle = False)

In [71]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value, env="main")
    break

torch.Size([32, 1, 28, 28])


### Line Plot

In [74]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data, env="main")

In [78]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

#### Line Update

In [79]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_376a9f394c64fe'

#### Multiple Line on Single Window

In [82]:
num = torch.Tensor(list(range(0,10)))
print(num.shape)
num = num.view(-1,1)
print(num.shape)
num = torch.cat((num,num),dim=1)
print(num.shape)

plt = vis.line(Y=torch.randn(10,2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


#### Line Information

In [83]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [84]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='test', legend = ['1번'], showlegend=True))

In [91]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번', '2번']))

### Make Func for Update Line

In [104]:
def loss_tracker(loss_plot, loss_value, num):
    
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append'
            )

In [105]:
plt = vis.line(Y=torch.Tensor(1).zero_())

In [106]:
for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

### Close the Window

In [107]:
vis.close(env="main")

''

# Lab 10.4 Custom Dataset

## 10.4.1 ImageFolder(1)

In [140]:
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [141]:
from matplotlib.pyplot import imshow
%matplotlib inline

In [144]:
trans = transforms.Compose([
    transforms.Resize((64,128))
])

train_data = torchvision.datasets.ImageFolder(root='dataset/custom/mouth',transform=trans)

In [146]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)
    
    if(label == 0):
        data.save('dataset/custom/newmouth/a/%d_%d.jpeg'%(num, label))
    elif(label == 1):
        data.save('dataset/custom/newmouth/e/%d_%d.jpeg'%(num, label))
    elif(label == 2):
        data.save('dataset/custom/newmouth/i/%d_%d.jpeg'%(num, label))
    elif(label == 3):
        data.save('dataset/custom/newmouth/o/%d_%d.jpeg'%(num, label))
    else:
        data.save('dataset/custom/newmouth/u/%d_%d.jpeg'%(num, label))
    

0 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348DA0> 0
1 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605C0> 0
2 <PIL.Image.Image image mode=RGB size=128x64 at 0x206381482B0> 0
3 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638360780> 0
4 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348DA0> 0
5 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638360780> 0
6 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348DA0> 0
7 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638496E10> 0
8 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348D30> 0
9 <PIL.Image.Image image mode=RGB size=128x64 at 0x206381482B0> 0
10 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605C0> 0
11 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348DA0> 0
12 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348F98> 0
13 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348DA0> 0
14 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605C0> 0
15 <P

122 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638360C50> 2
123 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638496F28> 2
124 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638496860> 2
125 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849AB70> 2
126 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849AC50> 2
127 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849AB70> 2
128 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348C50> 2
129 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849A390> 2
130 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849ACC0> 2
131 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348D30> 2
132 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605F8> 2
133 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348CC0> 2
134 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638496F60> 2
135 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605F8> 2
136 <PIL.Image.Image image mode=RGB size=128x64 

242 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605C0> 4
243 <PIL.Image.Image image mode=RGB size=128x64 at 0x2063849ADD8> 4
244 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605F8> 4
245 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638360908> 4
246 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383608D0> 4
247 <PIL.Image.Image image mode=RGB size=128x64 at 0x206383605F8> 4
248 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638348CC0> 4
249 <PIL.Image.Image image mode=RGB size=128x64 at 0x20638360908> 4


## 10.4.2 ImageFolder(2)

In [267]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import visdom

import torch.nn.init
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

vis = visdom.Visdom()
vis.close(env="main")

''

In [268]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num,
            Y=loss_value,
            win = loss_plot,
            update = 'append'
            )

In [269]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [270]:
#parameters
learning_rate = 0.00003
training_epochs = 60
batch_size = 8

In [271]:
trans = transforms.Compose([
    transforms.ToTensor()
])

train_data = torchvision.datasets.ImageFolder(root='dataset/custom/newmouth',transform=trans)

In [272]:
data_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True, num_workers = 2)

In [273]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(16*13*29,120,bias=True)
        self.fc2 = nn.Linear(120, 5,bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)           
    
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0],-1)
        out = self.relu(self.fc1(out))
        out = self.relu(self.fc2(out))
        return out

In [274]:
#testing
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)
test_out.size()

torch.Size([3, 5])

In [275]:
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)

In [276]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        out = net(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
    print('Epoch:{} cost={}'.format(epoch+1, avg_cost))
print('Learning Fin')

Epoch:1 cost=1.5933713912963867
Epoch:2 cost=1.5278387069702148
Epoch:3 cost=1.445021152496338
Epoch:4 cost=1.3446245193481445
Epoch:5 cost=1.2613375186920166
Epoch:6 cost=1.1571012735366821
Epoch:7 cost=1.0983394384384155
Epoch:8 cost=1.0276237726211548
Epoch:9 cost=1.0073121786117554
Epoch:10 cost=0.9745687246322632
Epoch:11 cost=0.9312992095947266
Epoch:12 cost=0.8917502164840698
Epoch:13 cost=0.8829463720321655
Epoch:14 cost=0.8444333076477051
Epoch:15 cost=0.8650643229484558
Epoch:16 cost=0.7936838865280151
Epoch:17 cost=0.7835711240768433
Epoch:18 cost=0.7950274348258972
Epoch:19 cost=0.7296151518821716
Epoch:20 cost=0.6602410078048706
Epoch:21 cost=0.6744866967201233
Epoch:22 cost=0.6110623478889465
Epoch:23 cost=0.6063286662101746
Epoch:24 cost=0.5788124203681946
Epoch:25 cost=0.5394876599311829
Epoch:26 cost=0.5299680829048157
Epoch:27 cost=0.5481258630752563
Epoch:28 cost=0.4954087734222412
Epoch:29 cost=0.48393332958221436
Epoch:30 cost=0.48442620038986206
Epoch:31 cost=0.48

In [277]:
torch.save(net.state_dict(), "./model/model.pth")

In [278]:
new_net = CNN().to(device)

In [279]:
new_net.load_state_dict(torch.load('./model/model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [280]:
trans = transforms.Compose([
    transforms.ToTensor()
])

test_data = torchvision.datasets.ImageFolder(root='dataset/custom/newmouth',transform=trans)

In [281]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [282]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        pred = net(imgs)
        
        correct_prediction = torch.argmax(pred, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())

Accuracy: 0.796000063419342


In [300]:
from PIL import Image

trans = transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])

image = Image.open('a.jpg')
x = trans(image)
x = x.view(1,3,64,128)
x = x.to(device)
pred = net(x)
name = ['a','e','i','o','u']
print(name[torch.argmax(pred, 1)])

i
